# **Chapter 19: Handling Advanced Web Scenarios**

---

## **19.1 Dynamic Content Handling**

### **Understanding Dynamic Web Applications**

Modern web applications rarely serve static HTML. Instead, they use **AJAX (Asynchronous JavaScript and XML)**, **WebSockets**, and **Server-Sent Events** to load content dynamically after the initial page load. This creates challenges for automation tools that expect immediate element availability.

**Common Dynamic Content Patterns:**
- **Infinite Scroll:** Content loads as user scrolls (social media feeds)
- **Lazy Loading:** Images/content load only when visible in viewport
- **AJAX Updates:** Partial page updates without full reload
- **Live Data:** Real-time updates via WebSockets (stock tickers, chat)
- **Progressive Disclosure:** Content appears based on user actions

### **Strategies for Testing Dynamic Content**

#### **Strategy 1: Explicit Waits for AJAX Completion**

```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class TestDynamicContent:
    """Handling AJAX and dynamic loading"""
    
    def test_ajax_loaded_content(self, driver):
        """Wait for AJAX content to load"""
        driver.get("https://example.com/async-content")
        
        # Initial state: loading indicator visible
        loading = driver.find_element(By.ID, "loading-spinner")
        assert loading.is_displayed()
        
        # Wait for content to load (loading disappears)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.invisibility_of_element_located((By.ID, "loading-spinner")))
        
        # Now content should be available
        content = driver.find_element(By.ID, "async-content")
        assert content.is_displayed()
        assert len(content.text) > 0
    
    def test_ajax_with_specific_text(self, driver):
        """Wait for specific text to appear"""
        driver.get("https://example.com/search")
        
        driver.find_element(By.ID, "search-box").send_keys("query")
        driver.find_element(By.ID, "search-btn").click()
        
        # Wait for results text to appear
        wait = WebDriverWait(driver, 10)
        results = wait.until(
            EC.presence_of_element_located((By.ID, "results"))
        )
        
        # Wait for specific content
        wait.until(
            EC.text_to_be_present_in_element(
                (By.ID, "result-count"), 
                "Found"
            )
        )
    
    def test_infinite_scroll(self, driver):
        """Test infinite scroll loading"""
        driver.get("https://example.com/feed")
        
        # Get initial item count
        initial_items = driver.find_elements(By.CLASS_NAME, "feed-item")
        initial_count = len(initial_items)
        
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for new items to load
        wait = WebDriverWait(driver, 10)
        wait.until(
            lambda d: len(d.find_elements(By.CLASS_NAME, "feed-item")) > initial_count
        )
        
        # Verify new items loaded
        new_items = driver.find_elements(By.CLASS_NAME, "feed-item")
        assert len(new_items) > initial_count
        
        # Verify content loaded (not just empty containers)
        for item in new_items[initial_count:]:
            assert len(item.text) > 0 or item.find_elements(By.TAG_NAME, "img")
    
    def test_lazy_loading_images(self, driver):
        """Verify images load when scrolled into view"""
        driver.get("https://example.com/gallery")
        
        # Get all image placeholders
        images = driver.find_elements(By.CSS_SELECTOR, "img[data-src]")
        assert len(images) > 0
        
        # Initially, images above fold should load, below should not
        viewport_height = driver.execute_script("return window.innerHeight")
        
        for img in images:
            location = img.location["y"]
            src = img.get_attribute("src")
            data_src = img.get_attribute("data-src")
            
            if location < viewport_height:
                # Should be loaded
                assert src is not None
                assert data_src is None or src == data_src
            else:
                # Should be lazy (data-src present, src empty/placeholder)
                pass  # Will verify after scroll
        
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        
        # Wait for lazy load
        time.sleep(2)  # Or better: wait for specific image
        
        # Verify bottom images now loaded
        bottom_img = images[-1]
        WebDriverWait(driver, 5).until(
            lambda d: bottom_img.get_attribute("src") is not None
        )
```

---

## **19.2 AJAX and Asynchronous Calls**

### **Understanding AJAX Patterns**

**AJAX (Asynchronous JavaScript and XML)** allows web pages to update asynchronously by exchanging data with a web server behind the scenes. This means a web page can be updated without reloading the entire page.

**Common AJAX Patterns:**
1. **Polling:** Client repeatedly asks server for updates
2. **Long Polling:** Client waits with connection open until data available
3. **Server-Sent Events (SSE):** Server pushes data to client
4. **WebSockets:** Bidirectional, persistent connection

### **Testing AJAX with Selenium**

```python
class TestAJAX:
    """Testing asynchronous operations"""
    
    def test_ajax_form_submission(self, driver):
        """Form submits via AJAX, page doesn't reload"""
        driver.get("https://example.com/contact")
        
        # Fill form
        driver.find_element(By.ID, "name").send_keys("Test")
        driver.find_element(By.ID, "message").send_keys("Hello")
        
        # Submit
        submit_btn = driver.find_element(By.ID, "submit")
        submit_btn.click()
        
        # Page should NOT reload (URL unchanged)
        assert "/contact" in driver.current_url
        
        # Wait for success message
        wait = WebDriverWait(driver, 10)
        success = wait.until(
            EC.presence_of_element_located((By.ID, "success-message"))
        )
        assert "sent successfully" in success.text.lower()
        
        # Form should be cleared or disabled
        name_field = driver.find_element(By.ID, "name")
        assert name_field.get_attribute("value") == "" or not name_field.is_enabled()
    
    def test_real_time_updates(self, driver):
        """WebSocket or SSE real-time data"""
        driver.get("https://example.com/live-dashboard")
        
        # Get initial value
        initial = driver.find_element(By.ID, "live-counter").text
        
        # Wait for update (real-time)
        wait = WebDriverWait(driver, 30)
        wait.until(
            lambda d: d.find_element(By.ID, "live-counter").text != initial
        )
        
        # Verify it changed
        updated = driver.find_element(By.ID, "live-counter").text
        assert updated != initial
    
    def test_ajax_error_handling(self, driver):
        """Graceful handling of AJAX failures"""
        driver.get("https://example.com/data-viewer")
        
        # Trigger request that will fail (network offline simulation)
        driver.execute_script("window.stop();")  # Stop all network
        
        # Try to load data
        driver.find_element(By.ID, "load-data").click()
        
        # Should show error message, not hang
        wait = WebDriverWait(driver, 5)
        error = wait.until(
            EC.presence_of_element_located((By.ID, "error-message"))
        )
        assert "failed" in error.text.lower() or "error" in error.text.lower()
        assert "retry" in error.text.lower() or driver.find_element(By.ID, "retry-btn")
    
    def test_concurrent_ajax_requests(self, driver):
        """Multiple simultaneous AJAX calls"""
        driver.get("https://example.com/dashboard")
        
        # Trigger multiple simultaneous requests
        driver.execute_script("""
            fetch('/api/data1');
            fetch('/api/data2');
            fetch('/api/data3');
        """)
        
        # Wait for all to complete
        wait = WebDriverWait(driver, 10)
        wait.until(
            lambda d: d.execute_script("""
                return document.querySelectorAll('.loading').length === 0;
            """)
        )
        
        # Verify all sections loaded
        sections = driver.find_elements(By.CSS_SELECTOR, ".data-section")
        assert len(sections) >= 3
        for section in sections:
            assert len(section.text) > 0
```

---

## **19.3 Frames and iFrames**

### **Understanding Frames**

**Frames** divide a browser window into multiple independent sections, each capable of loading its own HTML document. **iFrames (Inline Frames)** are embedded windows within a page.

**Testing Challenges:**
- Elements inside frames are not accessible from main document
- Must switch context to frame before interacting
- Nested frames require sequential switching
- Cross-origin frames have security restrictions

### **Frame Handling Strategies**

```python
class TestFrames:
    """Comprehensive frame handling"""
    
    def test_single_iframe(self, driver):
        """Basic iframe switching"""
        driver.get("https://example.com/iframe-page")
        
        # Switch to iframe by index (0-based)
        driver.switch_to.frame(0)
        
        # Now interact with elements inside iframe
        editor = driver.find_element(By.ID, "wysiwyg-editor")
        editor.send_keys("Text inside iframe")
        
        # Switch back to main content
        driver.switch_to.default_content()
        
        # Now can interact with main page elements
        main_button = driver.find_element(By.ID, "main-page-button")
        main_button.click()
    
    def test_iframe_by_name_or_id(self, driver):
        """Switch using frame name or id attribute"""
        driver.get("https://example.com/frames")
        
        # By name attribute
        driver.switch_to.frame("frame-name")
        
        # Or by id
        driver.switch_to.default_content()
        driver.switch_to.frame("frame-id")
        
        # Work inside frame
        driver.find_element(By.TAG_NAME, "button").click()
        
        driver.switch_to.default_content()
    
    def test_iframe_by_web_element(self, driver):
        """Most robust: Find frame element then switch"""
        driver.get("https://example.com/nested-frames")
        
        # Find iframe element using any locator
        iframe = driver.find_element(By.CSS_SELECTOR, "iframe[data-testid='content-frame']")
        
        # Switch to it
        driver.switch_to.frame(iframe)
        
        # Now interact
        content = driver.find_element(By.ID, "dynamic-content")
        assert content.is_displayed()
        
        driver.switch_to.default_content()
    
    def test_nested_frames(self, driver):
        """Handle frames within frames"""
        driver.get("https://example.com/nested-frames")
        
        # Structure: main -> frame1 -> frame2 -> content
        
        # Switch to outer frame
        driver.switch_to.frame("frame-outer")
        
        # Switch to inner frame (relative to current)
        driver.switch_to.frame("frame-inner")
        
        # Now at deepest level
        deep_content = driver.find_element(By.ID, "deep-content")
        deep_content.click()
        
        # Switch back one level
        driver.switch_to.parent_frame()
        # Now in frame-outer
        
        # Switch to main document
        driver.switch_to.default_content()
        # Now at top level
    
    def test_cross_origin_iframe_limitations(self, driver):
        """Understand Same-Origin Policy restrictions"""
        driver.get("https://example.com")
        
        # Assume iframe loads different origin (cross-origin)
        driver.switch_to.frame("cross-origin-frame")
        
        # Can interact with elements (click, type) if accessible
        # But CANNOT access content due to Same-Origin Policy
        
        try:
            # This will fail for cross-origin frames
            html = driver.execute_script("return document.body.innerHTML")
            print("Same origin - can access content")
        except Exception as e:
            print(f"Cross-origin restriction: {e}")
        
        # However, Selenium can still interact with visible elements
        # through WebDriver protocol (which operates at browser level)
        try:
            button = driver.find_element(By.TAG_NAME, "button")
            button.click()  # This usually works even cross-origin
        except:
            pass
        
        driver.switch_to.default_content()
    
    def test_iframe_size_and_visibility(self, driver):
        """Test iframe dimensions and scroll behavior"""
        driver.get("https://example.com/iframe-demo")
        
        iframe = driver.find_element(By.ID, "content-frame")
        
        # Check dimensions
        size = iframe.size
        assert size["width"] > 0
        assert size["height"] > 0
        
        # Check if visible (not hidden)
        assert iframe.is_displayed()
        
        # Scroll iframe content
        driver.switch_to.frame(iframe)
        
        # Scroll inside iframe
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Check if scrolled
        scroll_position = driver.execute_script("return window.pageYOffset;")
        assert scroll_position > 0
        
        driver.switch_to.default_content()
```

---

## **19.4 Pop-ups, Alerts, and Modals**

### **Browser Dialogs vs. Custom Modals**

**Browser Dialogs (Native):**
- `alert()` - OK only
- `confirm()` - OK/Cancel
- `prompt()` - Text input + OK/Cancel

**Custom Modals (HTML/CSS):**
- Div-based overlays
- Part of DOM, not native browser dialogs
- More styling flexibility

### **Handling Native Alerts**

```python
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class TestAlerts:
    """Native browser alert handling"""
    
    def test_simple_alert(self, driver):
        """Handle alert() - OK only"""
        driver.get("https://example.com/alerts")
        
        # Trigger alert
        driver.find_element(By.ID, "alert-btn").click()
        
        # Wait for alert
        wait = WebDriverWait(driver, 10)
        wait.until(EC.alert_is_present())
        
        # Switch to alert
        alert = driver.switch_to.alert
        
        # Verify text
        assert alert.text == "This is an alert message!"
        
        # Accept (click OK)
        alert.accept()
        
        # Verify alert closed
        # Should be back to main window
        assert driver.find_element(By.ID, "result").text == "Alert accepted"
    
    def test_confirmation_dialog(self, driver):
        """Handle confirm() - OK/Cancel"""
        driver.get("https://example.com/confirm")
        
        driver.find_element(By.ID, "delete-btn").click()
        
        # Wait for confirm dialog
        WebDriverWait(driver, 10).until(EC.alert_is_present())
        
        alert = driver.switch_to.alert
        assert "Are you sure" in alert.text
        
        # Dismiss (click Cancel)
        alert.dismiss()
        
        # Verify cancel action taken
        assert driver.find_element(By.ID, "status").text == "Deletion cancelled"
        
        # Now test OK path
        driver.find_element(By.ID, "delete-btn").click()
        alert = driver.switch_to.alert
        alert.accept()
        
        assert driver.find_element(By.ID, "status").text == "Item deleted"
    
    def test_prompt_dialog(self, driver):
        """Handle prompt() - Text input"""
        driver.get("https://example.com/prompt")
        
        driver.find_element(By.ID, "rename-btn").click()
        
        WebDriverWait(driver, 10).until(EC.alert_is_present())
        
        alert = driver.switch_to.alert
        
        # Send text to prompt
        alert.send_keys("New Name")
        
        # Accept
        alert.accept()
        
        # Verify
        assert driver.find_element(By.ID, "item-name").text == "New Name"
    
    def test_unexpected_alert_handling(self, driver):
        """Handle alerts that appear during other operations"""
        driver.get("https://example.com/unpredictable")
        
        try:
            # Perform operation that might trigger alert
            driver.find_element(By.ID, "risky-btn").click()
            
            # Try to continue with next operation
            driver.find_element(By.ID, "next-btn").click()
            
        except UnexpectedAlertPresentException:
            # Handle unexpected alert
            alert = driver.switch_to.alert
            print(f"Unexpected alert: {alert.text}")
            alert.dismiss()
            
            # Retry operation
            driver.find_element(By.ID, "next-btn").click()
```

### **Handling Custom Modals (HTML/CSS)**

```python
class TestCustomModals:
    """HTML-based modal dialogs"""
    
    def test_modal_open_close(self, driver):
        """Test modal dialog interactions"""
        driver.get("https://example.com/modals")
        
        # Modal should not be visible initially
        modal = driver.find_element(By.ID, "login-modal")
        assert not modal.is_displayed()
        
        # Open modal
        driver.find_element(By.ID, "open-login").click()
        
        # Wait for modal to appear
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(modal))
        
        assert modal.is_displayed()
        
        # Interact with modal content
        modal.find_element(By.ID, "username").send_keys("test")
        modal.find_element(By.ID, "password").send_keys("pass")
        modal.find_element(By.ID, "modal-submit").click()
        
        # Modal should close on success
        wait.until(EC.invisibility_of_element(modal))
        assert not modal.is_displayed()
    
    def test_modal_close_on_escape(self, driver):
        """Test keyboard accessibility"""
        driver.get("https://example.com/modals")
        
        driver.find_element(By.ID, "open-modal").click()
        modal = driver.find_element(By.ID, "my-modal")
        
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(modal))
        
        # Press Escape
        from selenium.webdriver.common.keys import Keys
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        
        # Modal should close
        wait.until(EC.invisibility_of_element(modal))
    
    def test_modal_outside_click(self, driver):
        """Click outside modal to close"""
        driver.get("https://example.com/modals")
        
        driver.find_element(By.ID, "open-modal").click()
        modal = driver.find_element(By.ID, "my-modal")
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of(modal))
        
        # Click on backdrop (outside modal content)
        # Usually the modal container, not the content div
        backdrop = driver.find_element(By.CSS_SELECTOR, ".modal-backdrop")
        backdrop.click()
        
        wait.until(EC.invisibility_of_element(modal))
    
    def test_nested_modals(self, driver):
        """Handle modal within modal"""
        driver.get("https://example.com/nested-modals")
        
        # Open first modal
        driver.find_element(By.ID, "open-level-1").click()
        modal1 = driver.find_element(By.ID, "modal-level-1")
        WebDriverWait(driver, 10).until(EC.visibility_of(modal1))
        
        # Open second modal from within first
        modal1.find_element(By.ID, "open-level-2").click()
        modal2 = driver.find_element(By.ID, "modal-level-2")
        WebDriverWait(driver, 10).until(EC.visibility_of(modal2))
        
        # Close inner modal first
        modal2.find_element(By.CLASS_NAME, "close").click()
        WebDriverWait(driver, 10).until(EC.invisibility_of_element(modal2))
        
        # Outer modal should still be open
        assert modal1.is_displayed()
        
        # Close outer
        modal1.find_element(By.CLASS_NAME, "close").click()
```

---

## **19.4 File Upload and Download**

### **File Upload Strategies**

```python
import os
import time

class TestFileOperations:
    """Testing file upload and download"""
    
    def test_simple_file_upload(self, driver):
        """Standard HTML file input"""
        driver.get("https://example.com/upload")
        
        # Find file input (must be <input type="file">)
        file_input = driver.find_element(By.ID, "file-upload")
        
        # Send absolute file path
        test_file = os.path.abspath("test_data/sample.pdf")
        file_input.send_keys(test_file)
        
        # Submit
        driver.find_element(By.ID, "upload-btn").click()
        
        # Verify upload success
        wait = WebDriverWait(driver, 10)
        success = wait.until(
            EC.presence_of_element_located((By.ID, "upload-success"))
        )
        assert "uploaded successfully" in success.text
    
    def test_drag_and_drop_upload(self, driver):
        """HTML5 drag and drop file upload"""
        driver.get("https://example.com/drag-upload")
        
        # Create JavaScript to simulate drag-drop
        # Since Selenium can't actually drag files from OS,
        # we use the drop event with DataTransfer
        
        test_file = os.path.abspath("test_data/image.png")
        
        # JavaScript to trigger drop event with file
        js_script = """
            var fileInput = document.createElement('input');
            fileInput.type = 'file';
            fileInput.style.display = 'none';
            document.body.appendChild(fileInput);
            
            var dropZone = arguments[0];
            var filePath = arguments[1];
            
            // Create DataTransfer object
            var dt = new DataTransfer();
            
            // Note: We can't actually create File objects from path in JS
            // This requires using input type=file or API
            // For real testing, use the input method or test API directly
            
            dropZone.dispatchEvent(new DragEvent('drop', {
                dataTransfer: dt
            }));
        """
        
        drop_zone = driver.find_element(By.ID, "drop-zone")
        driver.execute_script(js_script, drop_zone, test_file)
        
        # Alternative: Many apps have hidden file input for drag-drop
        file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file']")
        file_input.send_keys(test_file)
        
        # Verify
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "upload-complete")))
    
    def test_multiple_file_upload(self, driver):
        """Upload multiple files at once"""
        driver.get("https://example.com/multi-upload")
        
        file_input = driver.find_element(By.ID, "files-input")
        
        # Multiple files separated by \n (newline)
        files = [
            os.path.abspath("test_data/file1.pdf"),
            os.path.abspath("test_data/file2.pdf"),
            os.path.abspath("test_data/file3.jpg")
        ]
        
        file_input.send_keys("\n".join(files))
        
        # Verify all files listed
        file_list = driver.find_elements(By.CLASS_NAME, "file-item")
        assert len(file_list) == 3
        
        # Upload
        driver.find_element(By.ID, "upload-all").click()
        
        # Verify all uploaded
        wait = WebDriverWait(driver, 10)
        success_items = wait.until(
            lambda d: len(d.find_elements(By.CLASS_NAME, "upload-success")) == 3
        )
    
    def test_file_download(self, driver):
        """Verify file download functionality"""
        import os
        import glob
        
        # Setup download directory
        download_dir = os.path.abspath("downloads")
        os.makedirs(download_dir, exist_ok=True)
        
        # Configure Chrome for downloads
        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        }
        chrome_options.add_experimental_option("prefs", prefs)
        
        driver = webdriver.Chrome(options=chrome_options)
        
        try:
            driver.get("https://example.com/downloads")
            
            # Clean download folder first
            files_before = set(glob.glob(os.path.join(download_dir, "*")))
            
            # Click download
            driver.find_element(By.ID, "download-report").click()
            
            # Wait for download (poll for new file)
            import time
            max_wait = 30
            downloaded_file = None
            
            for _ in range(max_wait):
                files_after = set(glob.glob(os.path.join(download_dir, "*")))
                new_files = files_after - files_before
                
                if new_files:
                    downloaded_file = new_files.pop()
                    break
                
                time.sleep(1)
            
            assert downloaded_file is not None, "Download failed"
            
            # Verify file
            assert os.path.exists(downloaded_file)
            assert os.path.getsize(downloaded_file) > 0
            
            # Verify extension
            assert downloaded_file.endswith('.pdf') or downloaded_file.endswith('.csv')
            
        finally:
            driver.quit()
            # Cleanup
            import shutil
            if os.path.exists(download_dir):
                shutil.rmtree(download_dir)
    
    def test_download_with_api_verification(self, driver):
        """Verify download by checking API call"""
        driver.get("https://example.com/reports")
        
        # Intercept download request
        driver.execute_cdp_cmd('Network.enable', {})
        
        # Click download
        driver.find_element(By.ID, "export").click()
        
        # Get network logs
        logs = driver.get_log('performance')
        
        # Find download request
        download_url = None
        for log in logs:
            message = json.loads(log['message'])['message']
            if 'Network.responseReceived' in message.get('method', ''):
                url = message['params']['response']['url']
                if 'download' in url or 'export' in url:
                    download_url = url
                    break
        
        assert download_url is not None
        
        # Verify via API
        import requests
        response = requests.head(download_url)
        assert response.status_code == 200
        assert 'content-length' in response.headers
        assert int(response.headers['content-length']) > 0
```

---

## **19.5 Captchas and Anti-Bot Measures**

### **Strategies for Testing Protected Applications**

```python
class TestAntiBotMeasures:
    """Handling security measures in test environments"""
    
    def test_captcha_bypass_in_test_env(self, driver):
        """
        Strategy 1: Test environment bypass
        Many apps disable captcha in test/staging
        """
        driver.get("https://staging.example.com/register")
        
        # Staging may have hidden field to bypass
        try:
            bypass = driver.find_element(By.NAME, "captcha_bypass_token")
            bypass.send_keys("test-token-123")
        except NoSuchElementException:
            pass  # No bypass available
        
        # Or staging may auto-fill captcha
        try:
            captcha_field = driver.find_element(By.ID, "captcha-input")
            if captcha_field.get_attribute("value") == "":
                # If empty, may need to skip or use API
                pytest.skip("Captcha not bypassed in this environment")
        except:
            pass
    
    def test_captcha_api_solution(self, driver):
        """
        Strategy 2: Use test APIs to get captcha solution
        Some test environments expose solution via API
        """
        driver.get("https://example.com/register")
        
        # Get captcha image source
        captcha_img = driver.find_element(By.ID, "captcha-image")
        captcha_src = captcha_img.get_attribute("src")
        
        # In test env, might have API to get text
        import requests
        try:
            # Hypothetical test API
            response = requests.get(
                "https://test-api.example.com/captcha-solution",
                params={"image_url": captcha_src},
                headers={"X-Test-Key": "secret"}
            )
            solution = response.json()["text"]
            
            # Enter solution
            driver.find_element(By.ID, "captcha-input").send_keys(solution)
        except:
            pytest.skip("Captcha API unavailable")
    
    def test_alternative_authentication(self, driver):
        """
        Strategy 3: Use alternative auth that bypasses captcha
        Many apps have API tokens or test accounts
        """
        # Instead of UI login with captcha, use API to set session
        import requests
        
        # Login via API
        session = requests.Session()
        response = session.post(
            "https://api.example.com/auth",
            json={"username": "test", "password": "test"}
        )
        token = response.json()["token"]
        
        # Set cookie in browser to bypass login UI
        driver.get("https://example.com")  # Must visit domain first
        
        driver.add_cookie({
            "name": "session_token",
            "value": token,
            "domain": "example.com"
        })
        
        # Now access protected page without captcha
        driver.get("https://example.com/dashboard")
        assert "Dashboard" in driver.title
    
    def test_rate_limiting_behavior(self, driver):
        """
        Strategy 4: Test rate limiting without triggering captcha
        """
        driver.get("https://example.com/login")
        
        # Test that rate limiting exists
        for i in range(5):
            driver.find_element(By.ID, "username").send_keys(f"user{i}")
            driver.find_element(By.ID, "password").send_keys("wrong")
            driver.find_element(By.ID, "login-btn").click()
            
            # Clear for next attempt
            driver.find_element(By.ID, "username").clear()
            driver.find_element(By.ID, "password").clear()
        
        # After 5 attempts, should see rate limit message or captcha
        # In test env, might show specific message
        try:
            rate_msg = driver.find_element(By.CLASS_NAME, "rate-limit-message")
            assert "too many" in rate_msg.text.lower() or "wait" in rate_msg.text.lower()
        except NoSuchElementException:
            # Or captcha appeared
            try:
                captcha = driver.find_element(By.ID, "captcha-container")
                assert captcha.is_displayed()
            except:
                pass  # Neither appeared - might be test environment bypass
    
    def test_content_security_policy(self, driver):
        """Verify CSP headers prevent XSS"""
        driver.get("https://example.com")
        
        # Try to execute inline script (should be blocked by CSP)
        result = driver.execute_script("""
            try {
                // This might fail if CSP blocks inline scripts
                window.cspTest = 'executed';
                return window.cspTest;
            } catch(e) {
                return 'blocked: ' + e.message;
            }
        """)
        
        # Note: This test is limited as WebDriver scripts run in
        # privileged context. Better to check headers via API:
        import requests
        response = requests.get("https://example.com")
        csp = response.headers.get("Content-Security-Policy", "")
        
        assert "default-src" in csp or "script-src" in csp
        assert "'unsafe-inline'" not in csp  # Should not allow inline scripts
```

---

## **19.6 Shadow DOM Testing**

### **Understanding Shadow DOM**

**Shadow DOM** is a web standard that encapsulates DOM and CSS, creating a "shadow" tree attached to an element. It's used by Web Components to isolate styling and behavior.

**Testing Challenge:** Elements inside Shadow DOM are not accessible via standard DOM queries from the main document.

```
Shadow DOM Structure:
┌─────────────────────────────────────┐
│  Main Document (Light DOM)          │
│  ┌─────────────────────────────┐   │
│  │ <my-custom-element>         │   │
│  │   #shadow-root (closed/open)│   │
│  │   ┌─────────────────────┐ │   │
│  │   │ Shadow DOM            │ │   │
│  │   │ ┌─────────────────┐   │ │   │
│  │   │ │ <input id="x">  │   │ │   │
│  │   │ │ <button>        │   │ │   │
│  │   │ └─────────────────┘   │ │   │
│  │   └─────────────────────┘ │   │
│  │ </my-custom-element>      │   │
│  └─────────────────────────────┘   │
└─────────────────────────────────────┘

Standard querySelector cannot pierce shadow boundary
```

### **Shadow DOM Testing Strategies**

```python
class TestShadowDOM:
    """Testing Web Components and Shadow DOM"""
    
    def test_access_open_shadow_dom(self, driver):
        """Access elements in open shadow root"""
        driver.get("https://example.com/web-components")
        
        # Find shadow host
        shadow_host = driver.find_element(By.CSS_SELECTOR, "my-custom-input")
        
        # Access shadow root (JavaScript required for Selenium < 4)
        shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
        
        # Now find elements inside shadow DOM
        inner_input = shadow_root.find_element(By.CSS_SELECTOR, "input")
        inner_input.send_keys("text in shadow dom")
        
        # Or use JavaScript to pierce shadow DOM
        element = driver.execute_script("""
            return document.querySelector('my-custom-input')
                   .shadowRoot.querySelector('input');
        """)
        element.clear()
        element.send_keys("via JS")
    
    def test_selenium_4_shadow_dom(self, driver):
        """Selenium 4 has built-in shadow DOM support"""
        driver.get("https://example.com/shadow-components")
        
        # Find shadow host
        shadow_host = driver.find_element(By.CSS_SELECTOR, "user-profile")
        
        # Access shadow root (Selenium 4)
        shadow_root = shadow_host.shadow_root
        
        # Query within shadow DOM
        username = shadow_root.find_element(By.ID, "username")
        assert username.text == "TestUser"
        
        # Nested shadow DOM
        nested_component = shadow_root.find_element(By.CSS_SELECTOR, "avatar-image")
        nested_shadow = nested_component.shadow_root
        img = nested_shadow.find_element(By.TAG_NAME, "img")
        assert img.get_attribute("src") is not None
    
    def test_closed_shadow_dom(self, driver):
        """Closed shadow DOM (inaccessible from JavaScript)"""
        driver.get("https://example.com/closed-shadow")
        
        # Closed shadow DOM cannot be accessed via shadowRoot property
        # Must use workarounds or test via external behavior
        
        shadow_host = driver.find_element(By.CSS_SELECTOR, "secure-component")
        
        # Try to access (will return null for closed shadow)
        shadow_root = driver.execute_script(
            "return arguments[0].shadowRoot", shadow_host
        )
        assert shadow_root is None  # Closed shadow returns null
        
        # Strategy: Test via external API or behavior
        # Click host element and verify expected external behavior
        shadow_host.click()
        
        # Verify something happened in light DOM
        result = driver.find_element(By.ID, "action-result")
        assert result.is_displayed()
        
        # Or use Chrome DevTools Protocol to bypass (if available)
        # This requires CDP access and is browser-specific
    
    def test_shadow_dom_styling_isolation(self, driver):
        """Verify Shadow DOM CSS isolation"""
        driver.get("https://example.com/styled-components")
        
        # Styles in shadow DOM should not affect light DOM
        # and vice versa
        
        # Get computed style of element in shadow DOM
        host = driver.find_element(By.CSS_SELECTOR, "styled-button")
        shadow_root = driver.execute_script(
            "return arguments[0].shadowRoot", host
        )
        button = shadow_root.find_element(By.TAG_NAME, "button")
        
        # Check shadow DOM styles applied
        bg_color = button.value_of_css_property("background-color")
        # Should be defined in shadow DOM, not affected by page CSS
        
        # Verify light DOM styles don't penetrate
        page_bg = driver.find_element(By.TAG_NAME, "body")
        page_bg_color = page_bg.value_of_css_property("background-color")
        
        # Button should not inherit body background
        assert bg_color != page_bg_color
    
    def test_slot_content_projection(self, driver):
        """Test Web Components slots"""
        driver.get("https://example.com/slots-demo")
        
        # Component with slot
        # <my-card>
        #   <span slot="title">Card Title</span>
        #   <p slot="content">Card content here</p>
        # </my-card>
        
        card = driver.find_element(By.CSS_SELECTOR, "my-card")
        
        # Access slotted content (in light DOM, not shadow DOM)
        title = card.find_element(By.CSS_SELECTOR, "[slot='title']")
        assert title.text == "Card Title"
        
        content = card.find_element(By.CSS_SELECTOR, "[slot='content']")
        assert "Card content" in content.text
        
        # Verify slotted content renders in correct position
        # (Visual verification or computed style checks)
```

---

## **19.5 File Upload and Download (Advanced)**

### **Advanced Upload Scenarios**

```python
class TestAdvancedFileOperations:
    """Complex file handling scenarios"""
    
    def test_upload_with_progress_tracking(self, driver):
        """Monitor upload progress"""
        driver.get("https://example.com/upload")
        
        file_input = driver.find_element(By.ID, "file-input")
        large_file = os.path.abspath("test_data/50mb-file.zip")
        file_input.send_keys(large_file)
        
        # Monitor progress bar
        wait = WebDriverWait(driver, 60)
        
        # Wait for upload to start
        progress_bar = driver.find_element(By.ID, "progress-bar")
        
        # Wait for completion (progress 100% or success message)
        wait.until(
            lambda d: "100%" in d.find_element(By.ID, "progress-text").text 
            or d.find_element(By.ID, "upload-success").is_displayed()
        )
        
        # Verify success
        assert driver.find_element(By.ID, "upload-success").is_displayed()
    
    def test_drag_and_drop_file_upload(self, driver):
        """HTML5 drag and drop upload"""
        driver.get("https://example.com/drag-upload")
        
        drop_zone = driver.find_element(By.ID, "drop-zone")
        
        # Simulate drag-drop using JavaScript
        # (Selenium doesn't support native OS file drag)
        js_script = """
            var fileInput = document.createElement('input');
            fileInput.type = 'file';
            fileInput.style.display = 'none';
            document.body.appendChild(fileInput);
            
            // Create DataTransfer
            var dt = new DataTransfer();
            
            // Note: In real scenario, we'd need to populate files
            // This requires browser permissions or API interception
            
            var event = new DragEvent('drop', {
                dataTransfer: dt,
                bubbles: true,
                cancelable: true
            });
            
            arguments[0].dispatchEvent(event);
            
            return 'dropped';
        """
        
        # Alternative: Many apps have hidden file input for drag-drop
        file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file']")
        
        # Make visible if hidden
        driver.execute_script("arguments[0].style.display = 'block';", file_input)
        
        # Upload file
        test_file = os.path.abspath("test_data/document.pdf")
        file_input.send_keys(test_file)
        
        # Verify upload triggered
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "upload-complete")))
    
    def test_multiple_file_upload(self, driver):
        """Upload multiple files simultaneously"""
        driver.get("https://example.com/multi-upload")
        
        file_input = driver.find_element(By.ID, "multi-file-input")
        
        # Multiple files separated by newline
        files = [
            os.path.abspath("test_data/file1.pdf"),
            os.path.abspath("test_data/file2.jpg"),
            os.path.abspath("test_data/file3.png")
        ]
        
        file_input.send_keys("\n".join(files))
        
        # Verify all three listed
        file_list = driver.find_elements(By.CLASS_NAME, "file-list-item")
        assert len(file_list) == 3
        
        # Upload all
        driver.find_element(By.ID, "upload-all").click()
        
        # Wait for all to complete
        wait = WebDriverWait(driver, 30)
        wait.until(
            lambda d: len(d.find_elements(By.CLASS_NAME, "upload-success")) == 3
        )
    
    def test_file_download_verification(self, driver):
        """Verify downloaded file content"""
        import os
        import glob
        
        download_dir = os.path.abspath("downloads")
        os.makedirs(download_dir, exist_ok=True)
        
        # Setup Chrome download
        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        }
        chrome_options.add_experimental_option("prefs", prefs)
        
        driver = webdriver.Chrome(options=chrome_options)
        
        try:
            driver.get("https://example.com/reports")
            
            # Clear previous downloads
            for f in glob.glob(os.path.join(download_dir, "*")):
                os.remove(f)
            
            # Trigger download
            driver.find_element(By.ID, "download-report").click()
            
            # Wait for download (poll for file)
            downloaded_file = None
            for _ in range(30):
                files = glob.glob(os.path.join(download_dir, "*"))
                if files:
                    downloaded_file = files[0]
                    break
                time.sleep(1)
            
            assert downloaded_file is not None, "Download failed"
            
            # Verify file type
            assert downloaded_file.endswith('.pdf') or downloaded_file.endswith('.csv')
            
            # Verify not empty
            assert os.path.getsize(downloaded_file) > 0
            
            # Verify content (if CSV)
            if downloaded_file.endswith('.csv'):
                with open(downloaded_file, 'r') as f:
                    content = f.read()
                    assert "Report" in content or "Data" in content
                    assert len(content.split('\n')) > 1  # Has data rows
            
        finally:
            driver.quit()
            # Cleanup
            import shutil
            if os.path.exists(download_dir):
                shutil.rmtree(download_dir)
```

---

## **19.6 Captchas and Anti-Bot Measures**

### **Testing Strategies**

```python
class TestAntiBotHandling:
    """Strategies for testing protected applications"""
    
    def test_captcha_test_mode(self, driver):
        """
        Strategy: Many apps have test mode that disables captcha
        or uses test keys that always pass
        """
        # Look for test environment indicators
        driver.get("https://staging.example.com/register")
        
        # Check if captcha is test version
        try:
            captcha_container = driver.find_element(By.ID, "captcha-container")
            # Check for test-specific attributes
            is_test = driver.execute_script("""
                return arguments[0].getAttribute('data-test-mode') === 'true';
            """, captcha_container)
            
            if is_test:
                # May have auto-fill or bypass
                pass
        except:
            pass
        
        # Or check for hidden test token
        try:
            test_token = driver.find_element(By.NAME, "captcha_test_token")
            test_token.send_keys("test-token-value")
        except:
            pass
    
    def test_api_bypass_for_testing(self, driver):
        """
        Strategy: Use backend API to create state
        bypassing UI captcha entirely
        """
        import requests
        
        # Create user via API (no captcha)
        api_response = requests.post(
            "https://api-staging.example.com/users",
            headers={"X-Test-Api-Key": "secret"},
            json={
                "username": "testuser",
                "email": "test@example.com",
                "password": "testpass123"
            }
        )
        
        assert api_response.status_code == 201
        user_id = api_response.json()["id"]
        
        # Now login via UI (no captcha for existing users)
        driver.get("https://staging.example.com/login")
        driver.find_element(By.ID, "username").send_keys("testuser")
        driver.find_element(By.ID, "password").send_keys("testpass123")
        driver.find_element(By.ID, "login-btn").click()
        
        # Should login without captcha
        WebDriverWait(driver, 10).until(
            EC.url_contains("/dashboard")
        )
    
    def test_captcha_presence_verification(self, driver):
        """
        Strategy: At minimum, verify captcha is present in production
        but don't try to solve it
        """
        driver.get("https://example.com/register")
        
        # Verify captcha widget is present
        try:
            # Google reCAPTCHA
            recaptcha = driver.find_element(By.CLASS_NAME, "g-recaptcha")
            assert recaptcha.is_displayed()
            
            # Verify site key present (indicates configured)
            site_key = recaptcha.get_attribute("data-sitekey")
            assert site_key is not None and len(site_key) > 10
            
        except NoSuchElementException:
            try:
                # hCaptcha
                hcaptcha = driver.find_element(By.CLASS_NAME, "h-captcha")
                assert hcaptcha.is_displayed()
            except:
                # Custom captcha
                captcha_img = driver.find_element(By.ID, "captcha-image")
                assert captcha_img.is_displayed()
                
                input_field = driver.find_element(By.ID, "captcha-input")
                assert input_field.is_displayed()
        
        # Do not attempt to solve - just verify presence
        # In test environment, use bypass methods described above
```

---

## **19.7 Shadow DOM Testing (Deep Dive)**

### **Advanced Shadow DOM Penetration**

```python
class TestShadowDOMDeep:
    """Advanced Shadow DOM testing techniques"""
    
    def test_deeply_nested_shadow_dom(self, driver):
        """Multiple levels of shadow nesting"""
        driver.get("https://example.com/web-components")
        
        # Path: body -> app-shell -> user-card -> avatar-component -> img
        # Each arrow is a shadow boundary
        
        # Strategy: Execute script to pierce all boundaries
        avatar_img = driver.execute_script("""
            // Get app shell
            const app = document.querySelector('app-shell');
            if (!app || !app.shadowRoot) return null;
            
            // Get user card from shadow
            const card = app.shadowRoot.querySelector('user-card');
            if (!card || !card.shadowRoot) return null;
            
            // Get avatar from shadow
            const avatar = card.shadowRoot.querySelector('avatar-component');
            if (!avatar || !avatar.shadowRoot) return null;
            
            // Get image
            return avatar.shadowRoot.querySelector('img');
        """)
        
        assert avatar_img is not None
        assert avatar_img.get_attribute("src") is not None
    
    def test_shadow_dom_form_submission(self, driver):
        """Forms inside shadow DOM"""
        driver.get("https://example.com/shadow-form")
        
        # Form is inside shadow DOM
        shadow_host = driver.find_element(By.CSS_SELECTOR, "registration-form")
        
        # Access shadow root
        shadow_root = driver.execute_script(
            "return arguments[0].shadowRoot", shadow_host
        )
        
        # Fill form inside shadow
        shadow_root.find_element(By.NAME, "email").send_keys("test@example.com")
        shadow_root.find_element(By.NAME, "password").send_keys("password123")
        
        # Submit
        shadow_root.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
        
        # Verify (result might be in light DOM or shadow)
        driver.switch_to.default_content()
        success = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "registration-success"))
        )
        assert success.is_displayed()
    
    def test_slot_content_testing(self, driver):
        """Test Web Component slots"""
        driver.get("https://example.com/slots-demo")
        
        # Component: <my-card>
        # Template: <slot name="title"></slot>
        #           <slot name="content"></slot>
        
        card = driver.find_element(By.CSS_SELECTOR, "my-card")
        
        # Slotted content is in light DOM, not shadow DOM
        # It's projected into shadow DOM slot
        
        # Access slotted content via light DOM (standard)
        title = card.find_element(By.CSS_SELECTOR, "[slot='title']")
        assert title.text == "Expected Title"
        
        content = card.find_element(By.CSS_SELECTOR, "[slot='content']")
        assert "Expected content" in content.text
        
        # To see how it's rendered in shadow DOM (visual verification)
        # Take screenshot of component
        card.screenshot("card-component.png")
    
    def test_shadow_dom_event_handling(self, driver):
        """Events in shadow DOM bubble correctly"""
        driver.get("https://example.com/shadow-events")
        
        host = driver.find_element(By.CSS_SELECTOR, "event-component")
        shadow_root = driver.execute_script(
            "return arguments[0].shadowRoot", host
        )
        
        button = shadow_root.find_element(By.TAG_NAME, "button")
        
        # Click in shadow DOM
        button.click()
        
        # Event should bubble to light DOM or trigger side effect
        # Verify result in light DOM
        result = driver.find_element(By.ID, "event-result")
        assert "Button clicked" in result.text
    
    def test_closed_shadow_dom_workarounds(self, driver):
        """Testing closed shadow DOM (most difficult)"""
        driver.get("https://example.com/closed-shadow")
        
        # Closed shadow DOM: shadowRoot returns null
        # Cannot be accessed via JavaScript
        
        host = driver.find_element(By.CSS_SELECTOR, "closed-component")
        
        # This returns null for closed shadow
        shadow = driver.execute_script("return arguments[0].shadowRoot", host)
        assert shadow is None
        
        # Strategies for testing closed shadow DOM:
        
        # 1. Test via external API
        # If component exposes events or attributes
        
        # 2. Use browser extensions (not recommended for CI)
        
        # 3. Test behavior, not implementation
        # Interact with host element and verify results
        
        # Click host (event handled internally by shadow DOM)
        host.click()
        
        # Verify expected behavior in light DOM
        result = driver.find_element(By.ID, "component-result")
        assert result.is_displayed()
        
        # 4. Use Chrome DevTools Protocol (CDP) to pierce
        # Only works in Chrome and requires CDP access
        try:
            # This might work depending on browser/version
            shadow_root = driver.execute_cdp_cmd(
                'DOM.describeNode',
                {'objectId': host.id}
            )
            print("CDP access to shadow DOM")
        except:
            pass
```

---

## **19.8 Progressive Web App (PWA) Testing**

### **PWA Specific Features**

```python
class TestPWA:
    """Progressive Web App testing"""
    
    def test_service_worker_registration(self, driver):
        """Verify service worker is registered"""
        driver.get("https://pwa.example.com")
        
        # Check if service worker is registered
        sw_registered = driver.execute_script("""
            return navigator.serviceWorker.ready.then(function(registration) {
                return true;
            }).catch(function() {
                return false;
            });
        """)
        
        # Note: execute_script doesn't handle promises well
        # Better approach:
        result = driver.execute_async_script("""
            var callback = arguments[0];
            if ('serviceWorker' in navigator) {
                navigator.serviceWorker.getRegistration().then(function(reg) {
                    callback(reg ? 'registered' : 'not registered');
                });
            } else {
                callback('not supported');
            }
        """)
        
        assert result == "registered", f"Service worker: {result}"
    
    def test_offline_functionality(self, driver):
        """Test app works offline"""
        driver.get("https://pwa.example.com")
        
        # Wait for service worker to cache
        time.sleep(3)
        
        # Go offline using Chrome DevTools
        driver.execute_cdp_cmd('Network.emulateNetworkConditions', {
            'offline': True,
            'latency': 0,
            'downloadThroughput': 0,
            'uploadThroughput': 0
        })
        
        # Try to navigate to cached page
        driver.find_element(By.ID, "nav-offline-page").click()
        
        # Should still work (cached)
        assert driver.find_element(By.ID, "offline-content").is_displayed()
        
        # Try uncached resource (should fail gracefully)
        try:
            driver.find_element(By.ID, "fetch-fresh-data").click()
            # Should show offline message
            offline_msg = driver.find_element(By.CLASS_NAME, "offline-message")
            assert offline_msg.is_displayed()
        except:
            pass
        
        # Go back online
        driver.execute_cdp_cmd('Network.emulateNetworkConditions', {
            'offline': False,
            'latency': 20,
            'downloadThroughput': 1024 * 1024,
            'uploadThroughput': 512 * 1024
        })
    
    def test_add_to_home_screen_prompt(self, driver):
        """Test PWA install prompt"""
        driver.get("https://pwa.example.com")
        
        # Check for beforeinstallprompt event
        # This is tricky to test as it's browser-specific
        
        # Check manifest is present
        manifest_link = driver.find_element(By.CSS_SELECTOR, "link[rel='manifest']")
        manifest_url = manifest_link.get_attribute("href")
        assert manifest_url is not None
        
        # Verify manifest content via fetch
        import requests
        manifest = requests.get(manifest_url).json()
        
        assert "name" in manifest or "short_name" in manifest
        assert "start_url" in manifest
        assert "display" in manifest  # standalone, fullscreen, etc.
        assert "icons" in manifest
    
    def test_push_notifications(self, driver):
        """Test push notification permission and display"""
        driver.get("https://pwa.example.com")
        
        # Check notification permission
        permission = driver.execute_script("""
            return Notification.permission;
        """)
        
        # Should be 'default', 'granted', or 'denied'
        assert permission in ['default', 'granted', 'denied']
        
        # Request permission (if not already granted)
        if permission == 'default':
            driver.find_element(By.ID, "enable-notifications").click()
            
            # Handle browser permission dialog (if appears)
            try:
                WebDriverWait(driver, 5).until(EC.alert_is_present())
                alert = driver.switch_to.alert
                alert.accept()  # Or dismiss based on test
            except TimeoutException:
                pass  # Permission dialog handled by browser differently
        
        # Trigger push notification (if supported in test env)
        driver.find_element(By.ID, "send-test-notification").click()
        
        # Verify notification displayed (if we can access it)
        # Note: Browser notifications are OS-level, hard to automate
        # Usually verified manually or via screenshot
    
    def test_background_sync(self, driver):
        """Test background sync for offline actions"""
        driver.get("https://pwa.example.com")
        
        # Perform action
        driver.find_element(By.ID, "data-input").send_keys("Important data")
        
        # Go offline
        driver.set_network_conditions(
            offline=True,
            latency=0,
            download_throughput=0,
            upload_throughput=0
        )
        
        # Try to save (should queue for sync)
        driver.find_element(By.ID, "save-btn").click()
        
        # Should show "will sync when online"
        sync_msg = driver.find_element(By.CLASS_NAME, "sync-pending")
        assert sync_msg.is_displayed()
        
        # Go back online
        driver.set_network_conditions(
            offline=False,
            latency=20,
            download_throughput=1024000,
            upload_throughput=512000
        )
        
        # Wait for sync
        time.sleep(3)
        
        # Verify data synced
        driver.refresh()
        saved_data = driver.find_element(By.ID, "saved-data")
        assert "Important data" in saved_data.text
```

---

## **Chapter Summary**

### **Key Takeaways:**

1. **Dynamic Content:** Use explicit waits for AJAX, infinite scroll, and lazy loading. Never use `time.sleep()`. Monitor network activity or DOM mutations when possible.

2. **AJAX Testing:** Wait for specific elements or text to appear. Use `execute_async_script` for promises. Intercept network calls to verify API behavior.

3. **Frames/iFrames:** Always switch to frame context before interaction. Use `switch_to.frame()` with index, name, or WebElement. Return to default content when done. Handle nested frames sequentially.

4. **Alerts:** Use `switch_to.alert` for native dialogs. Handle `UnexpectedAlertPresentException`. Dismiss or accept based on test scenario. Custom HTML modals require standard element interaction.

5. **File Upload:** Use `send_keys()` with absolute path for standard inputs. For drag-drop, reveal hidden file inputs or use JavaScript events. Handle multiple files with newline separation.

6. **File Download:** Configure browser profile with download directory. Poll for file appearance. Verify file size and content. Use API verification as alternative.

7. **Captchas:** Never automate production captchas. Use test environment bypasses, API tokens, or visual testing modes. Test functionality around captcha rather than solving it.

8. **Shadow DOM:** Use `execute_script` to access shadowRoot for open shadows. Selenium 4 has `shadow_root` property. Closed shadows require behavioral testing or CDP. Test slots via light DOM content.

9. **PWA Testing:** Verify service worker registration. Test offline functionality using network emulation. Check manifest validity. Test add-to-home-screen prompts. Verify background sync and push notifications (where testable).

### **Common Pitfalls:**

- **Timing Issues:** Not waiting for dynamic content
- **Frame Context:** Forgetting to switch back to default content
- **File Paths:** Using relative paths instead of absolute for upload
- **Alert Handling:** Not handling unexpected alerts causing test crashes
- **Shadow DOM:** Trying to query shadow elements from light DOM
- **Download Verification:** Not waiting for download to complete

---

## **📖 Next Chapter: Chapter 20 - API Fundamentals**

Now that you've mastered advanced web UI scenarios, **Chapter 20** transitions to **API Testing**—the critical layer beneath the user interface.

In **Chapter 20**, you'll learn:

- **What is an API?:** Understanding Application Programming Interfaces, REST architecture, and web services
- **API Types:** REST vs. SOAP vs. GraphQL—architectural differences and testing approaches
- **HTTP Methods Deep Dive:** GET, POST, PUT, PATCH, DELETE with proper use cases and idempotency
- **Status Codes:** Complete guide to 1xx, 2xx, 3xx, 4xx, 5xx responses
- **Request/Response Structure:** Headers, body, content types, authentication
- **API Documentation:** Reading Swagger/OpenAPI specs, Postman collections
- **First API Tests:** Writing your first automated API calls with Python requests

**Why Chapter 20 is Critical:** Modern applications are API-driven. The UI is just a consumer of APIs. Testing APIs directly is faster, more stable, and catches bugs earlier than UI tests. This chapter bridges your UI testing skills to the backend layer.

**Continue to Chapter 20 to begin your journey into API testing—the foundation of modern test automation!**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='18. web_application_testing_types.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. api_testing/20. api_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
